# Setup

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import json
import numpy as np
import pandas as pd
import pvlib
import pytz
import folium
import geopy.distance
from dotenv import load_dotenv
import os

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
load_dotenv()
NREL_API_KEY = os.getenv("NREL_API_KEY")
NREL_API_EMAIL = os.getenv("NREL_API_EMAIL")

In [3]:
import sys
sys.path.append('/Users/andrebach/code/solmap')
%load_ext autoreload
%autoreload 2

In [4]:
import data_generation

# Run it

In [5]:
solar_weather_timeseries, solar_weather_metadata = pvlib.iotools.get_psm3(
    latitude=data_generation.geometry.LAT_SAN_FRANCISCO,
    longitude=data_generation.geometry.LON_SAN_FRANCISCO,
    names=2019,
    leap_day=False,
    attributes=data_generation.solar.SOLAR_WEATHER_ATTRIBUTES,
    map_variables=True,
    api_key=NREL_API_KEY,
    email=NREL_API_EMAIL,
)

In [6]:
solar_weather_timeseries.query("Month == 1 and Day == 1 and 7 <= Hour <= 15")

,Year,Month,Day,Hour,Minute,temp_air,temp_dew,dhi,dhi_clear,dni,dni_clear,ghi,ghi_clear,albedo,pressure,wind_direction,wind_speed
2019-01-01 07:30:00-08:00,2019,1,1,7,30,4.2,-1.7,10.0,10.0,153.0,153.0,11.0,11.0,0.1,1020.0,33.0,3.2
2019-01-01 08:30:00-08:00,2019,1,1,8,30,6.0,-1.3,38.0,34.0,641.0,698.0,148.0,153.0,0.1,1021.0,37.0,3.4
2019-01-01 09:30:00-08:00,2019,1,1,9,30,7.8,-2.1,47.0,47.0,851.0,851.0,313.0,313.0,0.1,1021.0,40.0,3.2
2019-01-01 10:30:00-08:00,2019,1,1,10,30,9.4,-3.1,57.0,57.0,913.0,913.0,437.0,437.0,0.1,1021.0,38.0,2.6
2019-01-01 11:30:00-08:00,2019,1,1,11,30,10.7,-4.2,60.0,60.0,942.0,942.0,508.0,508.0,0.1,1020.0,28.0,2.1
2019-01-01 12:30:00-08:00,2019,1,1,12,30,11.6,-4.9,62.0,62.0,943.0,943.0,521.0,521.0,0.1,1020.0,17.0,2.1
2019-01-01 13:30:00-08:00,2019,1,1,13,30,12.0,-4.9,60.0,60.0,920.0,920.0,473.0,473.0,0.1,1020.0,12.0,2.1
2019-01-01 14:30:00-08:00,2019,1,1,14,30,12.0,-4.6,59.0,56.0,807.0,856.0,353.0,368.0,0.1,1020.0,14.0,1.8
2019-01-01 15:30:00-08:00,2019,1,1,15,30,11.8,-2.3,64.0,47.0,523.0,734.0,189.0,222.0,0.1,1020.0,16.0,1.5


In [7]:
congressional_summary_df = data_generation.geometry.load_congresional_district_points(limit=10)

i = 0


In [8]:
generation_df = data_generation.solar.generate_values_at_points(congressional_summary_df)

i = 0
Saved to generation_2024_11_15T19_18_04_003266.csv


In [9]:
ncolors = 20
palette = sns.color_palette("flare_r", ncolors)
hex_colors = palette.as_hex()
vmin = generation_df.generation.min()
vmax = generation_df.generation.max()
vrange = vmax - vmin
generation_df["color"] = generation_df.generation.apply(
    lambda x: hex_colors[int((ncolors - 1) * (x - vmin) / vrange)]
)

In [10]:
m = folium.Map(
    location=(generation_df.lat.mean(), generation_df.lon.mean()),
    zoom_start=4,
    height=650,
    width=1083,
)
folium.TileLayer('cartodbdark_matter').add_to(m)
for i, row in generation_df.iterrows():
    folium.CircleMarker(
        location=[row.lat, row.lon],
        fill=True,
        stroke=False,
        fill_opacity=0.8,
        radius=6,
        color=row.color,
    ).add_to(m)
m

In [11]:
data_generation.solar.save_as_geojson(congressional_summary_df, generation_df)

Saved to solar_points_2024_11_15T19_18_04_065762.json


True

# End of notebook